In [ ]:
import pandas as pd
from sklearn import preprocessing
from kmodes.kprototypes import KPrototypes
import plotly.graph_objs as go   

In [8]:
df = pd.read_csv('data/RFM.csv')
# needed columns for the model input
continuous_cols = ['frequency', 'recency', 'T']
categorical_cols = ['Country']

In [19]:
# standardisation
x_1 = preprocessing.scale(df[continuous_cols])
x_1 = pd.DataFrame(x_1, columns=continuous_cols)  # Array to data frame

In [22]:
model_input = pd.concat([x_1, df[categorical_cols]], axis=1)
model_input

,frequency,recency,T,Country
0,-0.307808,0.988744,0.812994,Iceland
1,-0.307808,0.988744,0.812994,Iceland
2,-0.307808,0.988744,0.812994,Iceland
3,-0.307808,0.988744,0.812994,Iceland
4,-0.307808,0.988744,0.812994,Iceland
...,...,...,...,...
389871,-0.462247,-0.641331,-0.751623,United Kingdom
389872,-0.462247,-0.641331,-0.751623,United Kingdom
389873,-0.462247,-0.641331,-0.751623,United Kingdom
389874,-0.462247,-0.641331,-0.751623,United Kingdom


## Modelling

### K-Means
####      Parameters
 -      ----------

### K-Prototypes
####      Parameters
 -      ----------
 -      X : array-like, shape=[n_samples, n_features]
 -      categorical : Index of columns that contain categorical data

In [25]:
#Choosing optimal K
cost = []
for num_clusters in list(range(2,10)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao')
    kproto.fit_predict(model_input, categorical=[len(continuous_cols)])
    cost.append(kproto.cost_)

KeyboardInterrupt: 

In [ ]:
trace0 = go.Scatter(
    x = list(range(2, 10)),
    y = cost,
    mode = 'lines',
    name = 'Distortions'
)

data = [trace0]
# Edit the layout
layout = go.Layout(title = 'Cluster Measures vs Number of Clusters',
          xaxis = dict(title = 'Number of Clusters'),
          yaxis = dict(title = 'Cost'),
          )
fig = go.Figure(data=data, layout=layout)
fig.show()